In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from sklearn.utils import shuffle
from tqdm import tqdm_notebook
#https://pythonhosted.org/keras-tqdm/
import math
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop,Adam
import shutil


ModuleNotFoundError: No module named 'keras_preprocessing'

In [3]:
train_path = "../input/histopathologic-cancer-detection/train/"
test_path = "../input/histopathologic-cancer-detection/test/"

print('Training Images:', len(os.listdir(train_path)))
print('Testing Images: ', len(os.listdir(test_path)))


Training Images: 174464
Testing Images:  45561


In [4]:
train_data = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
train_data['label'].value_counts()


label
0    103878
1     70586
Name: count, dtype: int64

In [5]:
test_data = pd.read_csv("../input/histopathologic-cancer-detection/sample_submission.csv", dtype=str)


In [6]:
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174464 entries, 0 to 174463
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      174464 non-null  object
 1   label   174464 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.7+ MB


In [7]:
test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45561 entries, 0 to 45560
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45561 non-null  object
 1   label   45561 non-null  object
dtypes: object(2)
memory usage: 712.0+ KB


In [8]:
train_data.head()


,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [9]:
test_data.head()


,id,label
0,acfe80838488fae3c89bd21ade75be5c34e66be7,0
1,a1991e73a9b676faddd2bd47c39754b14d1eb923,0
2,94fa32b29cc1c00403176c0795fffa3cfaa0f20e,0
3,0b820b71670c039dd0a51333d1c919f471a9e940,0
4,4b7a73f1fe1dafe2ffb7d2c0b83107f060b8d693,0


In [10]:
train_data.id = train_data.id + '.tif'
test_data.id = test_data.id + '.tif'
print(train_data.head())


                                             id  label
0  f38a6374c348f90b587e046aac6079959adf3835.tif      0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77.tif      1
2  755db6279dae599ebb4d39a9123cce439965282d.tif      0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08.tif      0
4  068aba587a4950175d04c680d38943fd488d6a9d.tif      0


In [11]:
print(test_data.head())


                                             id label
0  acfe80838488fae3c89bd21ade75be5c34e66be7.tif     0
1  a1991e73a9b676faddd2bd47c39754b14d1eb923.tif     0
2  94fa32b29cc1c00403176c0795fffa3cfaa0f20e.tif     0
3  0b820b71670c039dd0a51333d1c919f471a9e940.tif     0
4  4b7a73f1fe1dafe2ffb7d2c0b83107f060b8d693.tif     0


In [12]:
train_data.shape


(174464, 2)

In [13]:
SAMPLE_SIZE = 10000
df_normal = train_data[train_data['label'] == 0].sample(SAMPLE_SIZE, random_state = 42)
df_cancer = train_data[train_data['label'] == 1].sample(SAMPLE_SIZE, random_state = 42)

# Join the two data frame, so that both cancer and normal got in one data frame, 
## Remember to shuffle the data set , to avaoid biasing
df_subset = pd.concat([df_normal, df_cancer], axis=0).reset_index(drop=True)

#shuffle the dataframe using shuffle 
from sklearn.utils import shuffle
train_data_subset = shuffle(df_subset)

train_data_subset.head()


,id,label
1619,224c9f976f20c0ea38c1eb1c05b0246a3fc8be5a.tif,0
2952,429cd74aedf3e13333b6976780f96e25531cc7de.tif,0
1556,085269f8abd0283feb8493a276ad794e06e2006d.tif,0
17856,0f1c401dbfb12be0f7307a2735365e880bb273e6.tif,1
17737,addbbb1268b800591b0fe11369b99d3c793dc45b.tif,1


In [14]:
train_data_subset.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1619 to 15149
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20000 non-null  object
 1   label   20000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 468.8+ KB


In [15]:
####We can now split the dataset in trian and spllit####

### Here we are split the data into TRAIN and VALIDATION ###
from sklearn.model_selection import train_test_split

def split_data(df_train):
        df_train, df_valid = train_test_split(df_train, test_size=0.02, random_state=42,
                                     stratify=df_train['label'])
        
        # We have to set the iindex as 'id', otherwise was giving trouble while uploadiung
        train_data_subset.set_index('id', inplace=True)
        
        train_list = list(df_train['id'])
        valid_list = list(df_valid['id'])
        
        return df_train, df_valid, train_list, valid_list
#Lets split it now###
df_train, df_valid, train_list, valid_list = split_data(train_data_subset)
print('df_train_shape', df_train.shape)
print('df_validation_shape', df_valid.shape)


df_train_shape (19600, 2)
df_validation_shape (400, 2)


In [16]:
df_train=df_train.astype(str)


In [17]:
df_valid=df_valid.astype(str)


In [18]:
df_valid.info()


<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 1659 to 19090
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      400 non-null    object
 1   label   400 non-null    object
dtypes: object(2)
memory usage: 9.4+ KB


In [19]:
df_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19600 entries, 14431 to 10021
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      19600 non-null  object
 1   label   19600 non-null  object
dtypes: object(2)
memory usage: 459.4+ KB


In [20]:
train_datagen = ImageDataGenerator(
       horizontal_flip=True,
       vertical_flip=True,
       brightness_range=[0.5, 1.5],
       fill_mode='reflect',                               
        rotation_range=15,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2)
        #validation_split=0.15)


NameError: name 'ImageDataGenerator' is not defined

In [21]:
validation_datagen = ImageDataGenerator(
    rescale=1./255)


NameError: name 'ImageDataGenerator' is not defined

In [22]:
test_datagen = ImageDataGenerator(
       #horizontal_flip=True,
       #vertical_flip=True,
       #brightness_range=[0.5, 1.5],
       #fill_mode='reflect',                               
        #rotation_range=15,
        rescale=1./255)
        #shear_range=0.2,
        #zoom_range=0.2)


NameError: name 'ImageDataGenerator' is not defined

In [23]:
tr_size = 19600
va_size = 400
bs = 64

tr_steps = math.ceil(tr_size / bs)
va_steps = math.ceil(va_size / bs)

#math.ceil() function returns the smallest integral value greater than the number. 
#If number is already integer, same number is returned.

train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_train,
    directory = train_path,
    x_col = "id",
    y_col = "label",
    #subset = "training",
    batch_size = bs,
    seed = 1,
    shuffle = True,
    class_mode = "categorical",
    target_size = (96,96))


NameError: name 'train_datagen' is not defined

In [24]:
valid_generator = validation_datagen.flow_from_dataframe(
    dataframe = df_valid,
    directory = train_path,
    x_col = "id",
    y_col = "label",
    #subset = "validation",
    batch_size = bs,
    seed = 1,
    shuffle = True,
    class_mode = "categorical",
    target_size = (96,96))


NameError: name 'validation_datagen' is not defined

In [25]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_data,
    directory = test_path,
    x_col = "id",
    y_col = None,
    batch_size = 32,
    seed = 1,
    shuffle = False,
    class_mode = None,
    target_size = (96,96))


NameError: name 'test_datagen' is not defined

In [26]:
def training_images(seed):
    np.random.seed(seed)
    train_generator.reset()
    imgs, labels = next(train_generator)
    tr_labels = np.argmax(labels, axis=1)
    
    plt.figure(figsize=(12,12))
    for i in range(16):
        text_class = labels[i]
        plt.subplot(4,4,i+1)
        plt.imshow(imgs[i,:,:,:])
        if(text_class[0] == 0):
            plt.text(0, -5, 'Positive', color='r')
        else:
            plt.text(0, -5, 'Negative', color='b')
        plt.axis('off')
    plt.show()

training_images(1)


NameError: name 'train_generator' is not defined

In [27]:
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = 3))
model.add(BatchNormalization())

model.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = 3))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size = 3))
model.add(BatchNormalization())

model.add(Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation = 'sigmoid'))
model.summary()


NameError: name 'Sequential' is not defined

In [28]:
epochs = 5


In [29]:
%%time

optimizer=Adam(learning_rate=0.000001,beta_1=0.9,beta_2=0.999,epsilon=1e-08)

model.compile(optimizer=optimizer,loss=['binary_crossentropy'],metrics=['accuracy'])

h4 = model.fit_generator(train_generator, steps_per_epoch=tr_steps, epochs=5, validation_data=valid_generator, validation_steps=va_steps, verbose=1)


NameError: name 'Adam' is not defined

In [30]:
model.save('cnn_v01.h4')


NameError: name 'model' is not defined

In [31]:
test_pred = model.predict_generator(test_generator)


NameError: name 'model' is not defined

In [32]:
print(test_pred[:5])


NameError: name 'test_pred' is not defined

In [33]:
test_filenames = test_generator.filenames
test_filenames[ :5]


NameError: name 'test_generator' is not defined

In [34]:
test_filenames = [x.split(".")[0] for x in test_filenames]


NameError: name 'test_filenames' is not defined

In [35]:
test_filenames[ :5]


NameError: name 'test_filenames' is not defined

In [36]:
len(test_filenames)


NameError: name 'test_filenames' is not defined

In [37]:
classes = list(np.argmax(test_pred, axis=1))


NameError: name 'test_pred' is not defined

In [38]:
classes[:5]


NameError: name 'classes' is not defined

In [39]:
submission = pd.DataFrame({'id':test_filenames,
     'label':classes
    })
submission.head()


NameError: name 'test_filenames' is not defined

In [40]:
submission.to_csv("submission.csv", index = False)


NameError: name 'submission' is not defined